In [13]:
import pandas as pd

**Limpeza de dados: PIB do Brasil**

In [14]:
pib_br = pd.read_csv("PIB 1995-2024.csv", sep=",")
pib_br = pd.DataFrame(pib_br)
pib_br['Data'] = pib_br['Data'].astype(str)
pib_br['Data'] = pib_br['Data'].str.replace('.', '/')
pib_br['Data'] = pd.to_datetime(pib_br['Data'], format= '%Y/%m')
pib_br = pib_br[(pib_br['Data'] >= '1995-01-01') & (pib_br['Data'] <= '2024-01-01')]
pib_br = pib_br.rename(columns={'PIB - R$ (milhões)  - Banco Central do Brasil, Boletim, Seção Atividade Econômica (Bacen / Boletim / Ativ. Ec.) - BM12_PIB12 - ':'PIB Brasil'})
pib_br['PIB Brasil'] = pd.to_numeric(pib_br['PIB Brasil'], errors='coerce')
pib_br['PIB Brasil'] = pib_br['PIB Brasil'].apply(lambda x: '{:.2f}'.format(x))

**Limpeza de dados: PIB dos EUA**

In [15]:
pib_usa= pd.read_csv("GDP 1992-2024.csv", sep=",")
pib_usa=pd.DataFrame(pib_usa)
pib_usa=pib_usa.rename(columns={'Unnamed: 0':'Data'})

pib_usa=pib_usa[['Data', 'Monthly Nominal GDP Index']]
pib_usa=pib_usa.dropna()

pib_usa['Data'] = pib_usa['Data'].str.replace(' - ', '/')
pib_usa['Data'] = pib_usa['Data'].str.replace(' ', '')
meses = {
  'Jan': '01', 'Feb': '02', 'Mar': '03', 'Apr': '04', 
  'May': '05', 'June': '06', 'July':'07', 'Jun': '06', 'Jul': '07', 'Aug': '08', 
  'Sep': '09', 'Oct': '10', 'Nov': '11', 'Dec': '12',
}
for mes in meses.keys():
  pib_usa['Data'] = pib_usa['Data'].str.replace(mes, meses[mes])

pib_usa['Data'] = pd.to_datetime(pib_usa['Data'], format='%Y/%m')
pib_usa = pib_usa[(pib_usa['Data'] >= '1995-01-01') & (pib_usa['Data'] <= '2024-01-01')]
pib_usa = pib_usa.rename(columns={'Monthly Nominal GDP Index':'PIB EUA'})
pib_usa['PIB EUA'] = pd.to_numeric(pib_usa['PIB EUA'], errors='coerce')
pib_usa['PIB EUA'] = pib_usa['PIB EUA'].apply(lambda x: '{:.2f}'.format(x))

**Limpeza de dados: IBOV**

In [16]:
ibov = pd.read_csv("BVSP 1993-2024.csv", sep=",")
ibov = pd.DataFrame(ibov)
ibov['Date'] = ibov['Date'].astype(str)
ibov['Date'] = ibov['Date'].str.replace('-', '/')
ibov['Date'] = pd.to_datetime(ibov['Date'], format='%Y/%m/%d')
ibov = ibov.dropna()
ibov.set_index('Date', inplace=True)
ibov = ibov.resample('MS').first()
ibov.reset_index(inplace=True)
ibov = ibov[['Date', 'Close']]
ibov = ibov.rename(columns={'Date':'Data'})
ibov = ibov.rename(columns={'Close':'Fechamento IBOV'})
ibov = ibov[(ibov['Data'] >= '1995-01-01') & (ibov['Data'] <= '2024-01-01')]
ibov['Fechamento IBOV'] = pd.to_numeric(ibov['Fechamento IBOV'], errors='coerce')
ibov['Fechamento IBOV'] = ibov['Fechamento IBOV'].apply(lambda x: '{:.2f}'.format(x))

**Limpeza de dados: S&P500**

In [17]:
sp500 = pd.read_csv("SP500 1993-2024.csv", sep=",")
sp500 = sp500.dropna()
sp500 = sp500[['Data','Fechamento*']]
sp500['Data'] = sp500['Data'].astype(str)
sp500['Data'] = sp500['Data'].str.replace(' de ', '/').str.replace(' ', '').str.replace('.', '')
meses = {
  'jan': '01', 'fev': '02', 'mar': '03', 'abr': '04', 
  'mai': '05', 'junho': '06', 'julho': '07', 'jun': '06', 'jul': '07', 'ago': '08', 
  'set': '09', 'out': '10', 'nov': '11', 'dez': '12',
}
for mes in meses.keys():
  sp500['Data'] = sp500['Data'].str.replace(mes, meses[mes])
sp500['Data'] = pd.to_datetime(sp500['Data'], format='%d/%m/%Y')
sp500 = sp500[(sp500['Data'] >= '1995-01-01') & (sp500['Data'] <= '2024-01-01')]
sp500.set_index('Data', inplace=True)
sp500.sort_values(by='Data', ascending=True, inplace=True)
sp500 = sp500.resample('MS').first()
sp500['Fechamento*'] = sp500['Fechamento*'].str.replace('.', '').str.replace(',', '.')
sp500 = sp500.rename(columns={'Fechamento*':'Fechamento S&P500'})
sp500['Fechamento S&P500'] = pd.to_numeric(sp500['Fechamento S&P500'], errors='coerce')
sp500['Fechamento S&P500'] = sp500['Fechamento S&P500'].apply(lambda x: '{:.2f}'.format(x))

In [18]:
dados = pd.merge(pib_usa, pib_br, on='Data')
dados = pd.merge(dados, ibov, on='Data')
dados = pd.merge(dados, sp500, on='Data')
dados